# 🤖 Agentes Autónomos para Automatización

Objetivo: construir agentes con LangGraph y AutoGen que automaticen tareas complejas de ingeniería de datos: debugging, optimización de queries, orquestación de pipelines.

- Duración: 120-150 min
- Dificultad: Alta
- Stack: LangChain, LangGraph, AutoGen

## 1. Agente simple con LangChain

In [ ]:
# pip install langchain langgraph openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain import hub

llm = ChatOpenAI(model='gpt-4', temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))

@tool
def get_table_row_count(table_name: str) -> str:
    """Devuelve el número de filas de una tabla."""
    # Simulación
    counts = {'ventas': 1250000, 'clientes': 50000, 'productos': 8500}
    return f'La tabla {table_name} tiene {counts.get(table_name, 0)} filas.'

@tool
def get_table_schema(table_name: str) -> str:
    """Devuelve el esquema de una tabla."""
    schemas = {
        'ventas': 'venta_id (BIGINT), fecha (DATE), producto_id (INT), total (DECIMAL)',
        'clientes': 'cliente_id (INT), nombre (VARCHAR), email (VARCHAR), ciudad (VARCHAR)'
    }
    return schemas.get(table_name, 'Tabla no encontrada')

tools = [get_table_row_count, get_table_schema]

# Prompt
prompt = hub.pull('hwchase17/openai-tools-agent')

# Crear agente
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({
    'input': '¿Cuántas filas tiene la tabla ventas y cuál es su esquema?'
})

print('\nRespuesta final:', result['output'])

## 2. Agente para debugging de SQL

In [ ]:
@tool
def validate_sql_syntax(query: str) -> str:
    """Valida sintaxis SQL."""
    # Simulación simple
    errors = []
    if 'SELCT' in query.upper():
        errors.append('Typo: SELCT debería ser SELECT')
    if query.count('(') != query.count(')'):
        errors.append('Paréntesis desbalanceados')
    return 'Errores: ' + ', '.join(errors) if errors else 'Sintaxis válida'

@tool
def suggest_sql_optimization(query: str) -> str:
    """Sugiere optimizaciones."""
    tips = []
    if 'SELECT *' in query.upper():
        tips.append('Evita SELECT *, especifica columnas')
    if 'WHERE' not in query.upper() and 'JOIN' in query.upper():
        tips.append('Considera agregar WHERE para filtrar resultados')
    return 'Sugerencias: ' + ', '.join(tips) if tips else 'Query está bien optimizada'

debug_tools = [validate_sql_syntax, suggest_sql_optimization]

debug_agent = create_openai_tools_agent(llm, debug_tools, prompt)
debug_executor = AgentExecutor(agent=debug_agent, tools=debug_tools, verbose=True)

broken_query = 'SELCT * FROM ventas JOIN productos'

result = debug_executor.invoke({
    'input': f'Analiza esta query y sugiere mejoras: {broken_query}'
})

## 3. LangGraph: workflow con estados

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

class ETLState(TypedDict):
    query: str
    validated: bool
    optimized: bool
    result: str
    errors: Annotated[list, operator.add]

def validate_node(state: ETLState) -> ETLState:
    """Nodo de validación."""
    if 'SELECT' not in state['query'].upper():
        state['errors'].append('Query no es SELECT')
        state['validated'] = False
    else:
        state['validated'] = True
    return state

def optimize_node(state: ETLState) -> ETLState:
    """Nodo de optimización."""
    if state['validated']:
        # Simulación
        state['query'] = state['query'].replace('SELECT *', 'SELECT id, nombre')
        state['optimized'] = True
    return state

def execute_node(state: ETLState) -> ETLState:
    """Nodo de ejecución."""
    if state['optimized']:
        state['result'] = f'Query ejecutada: {state["query"]}'
    else:
        state['result'] = 'Ejecución cancelada por errores'
    return state

# Crear grafo
workflow = StateGraph(ETLState)
workflow.add_node('validate', validate_node)
workflow.add_node('optimize', optimize_node)
workflow.add_node('execute', execute_node)

workflow.set_entry_point('validate')
workflow.add_edge('validate', 'optimize')
workflow.add_edge('optimize', 'execute')
workflow.add_edge('execute', END)

app = workflow.compile()

# Ejecutar
initial_state = {
    'query': 'SELECT * FROM ventas',
    'validated': False,
    'optimized': False,
    'result': '',
    'errors': []
}

final_state = app.invoke(initial_state)
print('Estado final:', final_state)

## 4. AutoGen: múltiples agentes colaborativos

In [ ]:
# pip install pyautogen
import autogen

config_list = [{
    'model': 'gpt-4',
    'api_key': os.getenv('OPENAI_API_KEY')
}]

# Agente 1: Data Engineer
data_engineer = autogen.AssistantAgent(
    name='DataEngineer',
    llm_config={'config_list': config_list},
    system_message='''
Eres un ingeniero de datos experto. Tu tarea es:
1. Diseñar pipelines ETL
2. Escribir queries SQL optimizadas
3. Proponer arquitecturas de datos
'''
)

# Agente 2: QA Engineer
qa_engineer = autogen.AssistantAgent(
    name='QAEngineer',
    llm_config={'config_list': config_list},
    system_message='''
Eres un ingeniero de calidad. Tu tarea es:
1. Revisar código y queries
2. Identificar bugs y problemas de rendimiento
3. Sugerir tests
'''
)

# Agente 3: User Proxy (humano)
user_proxy = autogen.UserProxyAgent(
    name='User',
    human_input_mode='NEVER',
    max_consecutive_auto_reply=0
)

# Conversación
task = '''
Diseña un pipeline que:
1. Extraiga datos de API de ventas
2. Transforme (agregue por día)
3. Cargue en Redshift
Luego revisa el diseño.
'''

# Iniciar chat grupal
groupchat = autogen.GroupChat(
    agents=[user_proxy, data_engineer, qa_engineer],
    messages=[],
    max_round=5
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={'config_list': config_list})

user_proxy.initiate_chat(manager, message=task)

## 5. Agente para monitoreo de data quality

In [ ]:
@tool
def check_null_rate(table: str, column: str) -> str:
    """Verifica tasa de nulos."""
    # Simulación
    rates = {('ventas', 'total'): 0.5, ('clientes', 'email'): 15.2}
    rate = rates.get((table, column), 0)
    return f'Tasa de nulos en {table}.{column}: {rate}%'

@tool
def check_duplicates(table: str) -> str:
    """Verifica duplicados."""
    dup_counts = {'ventas': 120, 'clientes': 5}
    count = dup_counts.get(table, 0)
    return f'Duplicados en {table}: {count}'

quality_tools = [check_null_rate, check_duplicates]

quality_agent = create_openai_tools_agent(llm, quality_tools, prompt)
quality_executor = AgentExecutor(agent=quality_agent, tools=quality_tools, verbose=True)

quality_result = quality_executor.invoke({
    'input': 'Revisa la calidad de la tabla ventas: nulos y duplicados'
})

## 6. Agente reactivo con memoria

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

agent_with_memory = create_openai_tools_agent(llm, tools, prompt)
executor_with_memory = AgentExecutor(
    agent=agent_with_memory,
    tools=tools,
    memory=memory,
    verbose=True
)

# Conversación
executor_with_memory.invoke({'input': 'Cuántas filas tiene ventas?'})
executor_with_memory.invoke({'input': 'Y cuál es su esquema?'})  # Contexto previo

## 7. Buenas prácticas

- **Tools específicas**: cada tool debe hacer UNA cosa bien.
- **Validación de outputs**: verifica respuestas antes de acciones críticas.
- **Human-in-the-loop**: aprobación humana para operaciones destructivas.
- **Observabilidad**: loggea cada decisión del agente.
- **Fallback**: maneja errores de LLM o tools.
- **Testing**: prueba agentes con casos edge.
- **Costos**: limita llamadas y usa modelos pequeños cuando posible.

## 8. Ejercicios

1. Crea un agente que automatice la investigación de incidentes en pipelines.
2. Construye un agente que genere data quality reports diarios.
3. Implementa un sistema multi-agente para code review de PRs.
4. Desarrolla un agente que optimice queries en producción automáticamente.